In [1]:
import os

os.environ["PYTHONUNBUFFERED"] = "1"
os.environ["JAVA_HOME"] = "/opt/homebrew/opt/openjdk@17"
os.environ["SPARK_HOME"] = "/opt/homebrew/Cellar/apache-spark/4.0.0/libexec"
os.environ["PATH"] = os.environ["JAVA_HOME"] + "/bin:" + os.environ["SPARK_HOME"] + "/bin:" + os.environ["PATH"]

In [2]:
os.environ["PATH"]

'/opt/homebrew/opt/openjdk@17/bin:/opt/homebrew/Cellar/apache-spark/4.0.0/libexec/bin:/Users/aleksejkitajskij/Desktop/goit_spark/venv/bin:/opt/homebrew/opt/apache-spark/libexec/bin:/opt/anaconda3/bin:/opt/anaconda3/condabin:/Users/aleksejkitajskij/fam_terminal:/opt/homebrew/opt/openjdk@17/bin:/opt/homebrew/bin:/opt/homebrew/sbin:/Library/Frameworks/Python.framework/Versions/3.12/bin:/Library/Frameworks/Python.framework/Versions/3.10/bin:/Library/Frameworks/Python.framework/Versions/3.11/bin:/usr/local/bin:/System/Cryptexes/App/usr/bin:/usr/bin:/bin:/usr/sbin:/sbin:/var/run/com.apple.security.cryptexd/codex.system/bootstrap/usr/local/bin:/var/run/com.apple.security.cryptexd/codex.system/bootstrap/usr/bin:/var/run/com.apple.security.cryptexd/codex.system/bootstrap/usr/appleinternal/bin:/usr/local/share/dotnet:~/.dotnet/tools'

In [3]:
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, TimestampType

In [4]:
# Створення SparkSession
spark = SparkSession.builder.appName("CSVStream").getOrCreate()

Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
25/08/25 18:34:34 WARN Utils: Your hostname, Aleksejs-MacBook-Air.local, resolves to a loopback address: 127.0.0.1; using 192.168.0.226 instead (on interface en0)
25/08/25 18:34:34 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/08/25 18:34:35 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [5]:
# Визначення схеми для CSV-файлу
schema = StructType([
    StructField("id", IntegerType(), True),
    StructField("event_time", TimestampType(), True),
    StructField("value", StringType(), True)
])

# append

In [6]:
# Читання потокових даних із CSV-файлу
csvDF = spark.readStream \
    .option("sep", ",") \
    .option("header", True) \
    .option("maxFilesPerTrigger", 1) \
    .schema(schema) \
    .csv("data/data_for_streaming")  # Папка для моніторингу нових файлів

# Старт стримінгу й виведення результатів на екран
query = (csvDF.writeStream
         .trigger(availableNow=True)
         # .trigger(processingTime='10 seconds')
         .outputMode("append")
         .format("console")
         .start()
         )

# Чекаємо кінця роботи стримінгу
query.awaitTermination()

25/08/25 18:34:37 WARN ResolveWriteToStream: Temporary checkpoint location created which is deleted normally when the query didn't fail: /private/var/folders/7x/q4zhf1wx08z6l2f4bqz5b8dc0000gn/T/temporary-ae70e0cf-9b7c-4f05-8c8a-1092c1d75441. If it's required to delete it under any circumstances, please set spark.sql.streaming.forceDeleteTempCheckpointLocation to true. Important to know deleting temp checkpoint folder is best effort.
25/08/25 18:34:37 WARN ResolveWriteToStream: spark.sql.adaptive.enabled is not supported in streaming DataFrames/Datasets and will be disabled.


-------------------------------------------
Batch: 0
-------------------------------------------
+---+-------------------+-----+
| id|         event_time|value|
+---+-------------------+-----+
|  1|2024-06-09 12:02:00|  cat|
|  1|2024-06-09 12:02:00|  dog|
+---+-------------------+-----+

-------------------------------------------
Batch: 1
-------------------------------------------
+---+-------------------+-----+
| id|         event_time|value|
+---+-------------------+-----+
|  1|2024-06-09 12:03:00|  dog|
|  1|2024-06-09 12:03:00|  dog|
+---+-------------------+-----+



# update

In [8]:
csvDF = spark.readStream \
    .option("sep", ",") \
    .option("header", True) \
    .option("maxFilesPerTrigger", 1) \
    .schema(schema) \
    .csv("data/data_for_streaming")

# Групування даних за полем 'value' і підрахунок кількості
countDF = csvDF.groupBy("value").count()

# Старт стримінгу та виведення результатів на екран
query = (countDF.writeStream
         .trigger(availableNow=True)
         .outputMode("update")
         # .outputMode("update")
         .format("console")
         .start()
         )

# Чекаємо кінця роботи стримінгу
query.awaitTermination()

25/08/25 18:43:10 WARN ResolveWriteToStream: Temporary checkpoint location created which is deleted normally when the query didn't fail: /private/var/folders/7x/q4zhf1wx08z6l2f4bqz5b8dc0000gn/T/temporary-6831bc60-35c6-40bd-bed0-876e46aea0e6. If it's required to delete it under any circumstances, please set spark.sql.streaming.forceDeleteTempCheckpointLocation to true. Important to know deleting temp checkpoint folder is best effort.
25/08/25 18:43:10 WARN ResolveWriteToStream: spark.sql.adaptive.enabled is not supported in streaming DataFrames/Datasets and will be disabled.


-------------------------------------------
Batch: 0
-------------------------------------------
+-----+-----+
|value|count|
+-----+-----+
|  dog|    1|
|  cat|    1|
+-----+-----+



-------------------------------------------
Batch: 1
-------------------------------------------
+-----+-----+
|value|count|
+-----+-----+
|  dog|    3|
+-----+-----+



# complete

In [9]:
csvDF = spark.readStream \
    .option("sep", ",") \
    .option("header", True) \
    .option("maxFilesPerTrigger", 1) \
    .schema(schema) \
    .csv("data/data_for_streaming")

# Групування даних за полем 'value' і підрахунок кількості
countDF = csvDF.groupBy("value").count()

# Старт стримінгу та виведення результатів на екран
query = (countDF.writeStream
         .trigger(availableNow=True)
         .outputMode("complete")
         # .outputMode("update")
         .format("console")
         .start()
         )

# Чекаємо кінця роботи стримінгу
query.awaitTermination()

25/08/25 18:43:21 WARN ResolveWriteToStream: Temporary checkpoint location created which is deleted normally when the query didn't fail: /private/var/folders/7x/q4zhf1wx08z6l2f4bqz5b8dc0000gn/T/temporary-ed6914b5-857b-4095-adae-d20dc6f0ff94. If it's required to delete it under any circumstances, please set spark.sql.streaming.forceDeleteTempCheckpointLocation to true. Important to know deleting temp checkpoint folder is best effort.
25/08/25 18:43:21 WARN ResolveWriteToStream: spark.sql.adaptive.enabled is not supported in streaming DataFrames/Datasets and will be disabled.


-------------------------------------------
Batch: 0
-------------------------------------------
+-----+-----+
|value|count|
+-----+-----+
|  dog|    1|
|  cat|    1|
+-----+-----+



-------------------------------------------
Batch: 1
-------------------------------------------
+-----+-----+
|value|count|
+-----+-----+
|  dog|    3|
|  cat|    1|
+-----+-----+

